In [1]:
from pathlib import Path
import logging
import os
import shutil
from datetime import datetime, timedelta
import bz2
from urllib import request
from urllib.error import HTTPError
from tqdm.notebook import tqdm
import glob
import xarray as xr
from scripts.editors import triple_position_number

icon_storage = Path('../geo_data/icon_results')


coord_limits = {'max_lon': 179., 'min_lon': 19.,
                'max_lat': 72., 'min_lat': 40.}

In [4]:
'tmax_2m'.upper()

'TMAX_2M'

In [9]:
icon_storage = 'https://opendata.dwd.de/weather/nwp/icon/grib'
# possible times ['00', '06', '12', '18']
icon_times = ['00']
icon_variables = ['tmax_2m', 'tmin_2m', 'tot_prec', 'alb_rad']
icon_type = 'icon_global_icosahedral_single-level'
# get today variables
today_date = datetime.today().strftime('%Y%m%d')

icon_model_time = [triple_position_number(val)
                   for val in range(0, 181)]
examples = dict()
# download every evailable value for modeled time
for time in tqdm(icon_times, 'time parsing..'):
    for var in icon_variables:
        print(f'\nDownload for {var} ..')
        examples[var] = list()
        for m_time in icon_model_time:
            storage_path = Path(f'./data/icon_bz2/{time}/{var}')
            storage_path.mkdir(exist_ok=True, parents=True)

            icon_time_tag = f'{today_date}{time}_{m_time}'
            icon_link = f'{icon_storage}/{time}/{var}/{icon_type}'
            
            dw_url = f'{icon_link}_{icon_time_tag}_{var.upper()}.grib2.bz2'
        #     try:
        #         request.urlretrieve(
        #             dw_url,
        #             f'{storage_path}/{icon_type}_{icon_time_tag}_{var.upper()}.grib2.bz2')
        #     except HTTPError:
        #         continue
        # print(f'Download for {var} has been finished !')
            examples[var].append(dw_url)


time parsing..:   0%|          | 0/1 [00:00<?, ?it/s]


Download for tmax_2m ..

Download for tmin_2m ..

Download for tot_prec ..

Download for alb_rad ..


In [ ]:
import schedule
import time

def job(t):
    print "I'm working...", t
    return

schedule.every().day.at("01:00").do(job,'It is 01:00')

while True:
    schedule.run_pending()
    time.sleep(60) # wait one minute

In [10]:
examples['tmax_2m']

['https://opendata.dwd.de/weather/nwp/icon/grib/00/tmax_2m/icon_global_icosahedral_single-level_2023021700_000_TMAX_2M.grib2.bz2',
 'https://opendata.dwd.de/weather/nwp/icon/grib/00/tmax_2m/icon_global_icosahedral_single-level_2023021700_001_TMAX_2M.grib2.bz2',
 'https://opendata.dwd.de/weather/nwp/icon/grib/00/tmax_2m/icon_global_icosahedral_single-level_2023021700_002_TMAX_2M.grib2.bz2',
 'https://opendata.dwd.de/weather/nwp/icon/grib/00/tmax_2m/icon_global_icosahedral_single-level_2023021700_003_TMAX_2M.grib2.bz2',
 'https://opendata.dwd.de/weather/nwp/icon/grib/00/tmax_2m/icon_global_icosahedral_single-level_2023021700_004_TMAX_2M.grib2.bz2',
 'https://opendata.dwd.de/weather/nwp/icon/grib/00/tmax_2m/icon_global_icosahedral_single-level_2023021700_005_TMAX_2M.grib2.bz2',
 'https://opendata.dwd.de/weather/nwp/icon/grib/00/tmax_2m/icon_global_icosahedral_single-level_2023021700_006_TMAX_2M.grib2.bz2',
 'https://opendata.dwd.de/weather/nwp/icon/grib/00/tmax_2m/icon_global_icosahedral_

In [ ]:
def download_bz2(icon_storage: str,
                 icon_times: list,
                 icon_variables: list,
                 icon_type):
    
    icon_model_time = [triple_position_number(val)
                   for val in range(0, 181)]
    # download every evailable value for modeled time
    for time in tqdm(icon_times, 'time parsing..'):
        for var in icon_variables:
            print(f'\nDownload for {var} ..')
            for m_time in icon_model_time:
                storage_path = Path(f'./data/icon_bz2/{time}/{var}')
                storage_path.mkdir(exist_ok=True, parents=True)

                icon_time_tag = f'{today_date}{time}_{m_time}'
                icon_link = f'{icon_storage}/{time}/{var}/{icon_type}'
                
                dw_url = f'{icon_link}_{icon_time_tag}_{var.upper()}.grib2.bz2'
                try:
                    request.urlretrieve(
                        dw_url,
                        f'{storage_path}/{icon_type}_{icon_time_tag}_{var.upper()}.grib2.bz2')
                except HTTPError:
                    continue
            print(f'Download for {var} has been finished !')

In [7]:
# unarchive bz2 files in each folder with correspond variable
for time in icon_times:
    for var in icon_variables:
        print(f'\nUnarchiving for {var} ..')
        files = glob.glob(f'./data/icon_bz2/{time}/{var}/*.bz2')
        
        for filename in files:
            f_name = filename.split('/')[-1][:-4]

            file_folder = Path(f'./data/icon_grib2/{time}/{var}')
            file_folder.mkdir(exist_ok=True, parents=True)

            fin_file = f'{file_folder}/{f_name}'

            with open(fin_file, 'wb') as nw_f, bz2.BZ2File(filename, 'rb') as f:
                for data in iter(lambda: f.read(100 * 1024), b''):
                    nw_f.write(data)
        print(f'Unarchive for {var} has been finished !')


Unarchiving for tmax_2m ..
Unarchive for tmax_2m has been finished !

Unarchiving for tmin_2m ..
Unarchive for tmin_2m has been finished !

Unarchiving for tot_prec ..
Unarchive for tot_prec has been finished !

Unarchiving for alb_rad ..
Unarchive for alb_rad has been finished !


In [3]:
desc_f = './data/static_icon/grid_world_0125.txt'
weight_f = './data/static_icon/weights_icon_world.nc'

for time in tqdm(icon_times, 'time parsing..'):
    for var in icon_variables:
        print(f'\nNetcdf conversion for {var} ..')

        files = glob.glob(f'./data/icon_grib2/{time}/{var}/*.grib2')
        fin_folder = Path(f'./data/icon_nc/{time}/{var}')
        fin_folder.mkdir(exist_ok=True, parents=True)

        for filename in files:

            out_f = filename.split('/')[-1][:-6]
            fin_f = f'{fin_folder}/{out_f}'

            os.system(
                f'cdo -s -f nc remap,{desc_f},{weight_f} {filename} {fin_f}.nc')
        print(f'Netcdf conversion for {var} has been finished !')


time parsing..:   0%|          | 0/1 [00:00<?, ?it/s]


Netcdf conversion for tmax_2m ..
Netcdf conversion for tmax_2m has been finished !

Netcdf conversion for tmin_2m ..
Netcdf conversion for tmin_2m has been finished !

Netcdf conversion for tot_prec ..
Netcdf conversion for tot_prec has been finished !

Netcdf conversion for alb_rad ..
Netcdf conversion for alb_rad has been finished !


In [4]:
def from_icon_to_ru_nc(path_to_nc: Path,
                       var: str,
                       icon_time: str,
                       path_to_result: Path):
    """_summary_

    Args:
        path_to_nc (Path): _description_
        var (str): _description_
        icon_time (str): _description_
        path_to_result (Path): _description_
    """

    if var == 'tot_prec':
        ds_to_save = xr.open_mfdataset(
            glob.glob(
                f'{path_to_nc}/{icon_time}/{var}/*.nc')).resample(
            time='1d').sum()
    else:
        ds_to_save = xr.open_mfdataset(
            glob.glob(
                f'{path_to_nc}/{icon_time}/{var}/*.nc')).resample(
            time='1d').mean()

    ds_to_save = ds_to_save.where(
        ds_to_save.lon <= coord_limits['max_lon'], drop=True).where(
        ds_to_save.lon >= coord_limits['min_lon'], drop=True).where(
        ds_to_save.lat <= coord_limits['max_lat'], drop=True).where(
        ds_to_save.lat >= coord_limits['min_lat'], drop=True)
        
    today_date = datetime.today().strftime('%Y-%m-(%d')
    day_step = (datetime.today() + timedelta(7)).strftime('-%d)')
    
    
    forecast_period = today_date + day_step
    
    path_to_result = Path(f'{path_to_result}/{var}/')
    path_to_result.mkdir(exist_ok=True, parents=True)
    
    ds_to_save.to_netcdf(f'{path_to_result}/{var}_{forecast_period}.nc')
    
    return ds_to_save

In [12]:

shutil.rmtree('./data/icon_bz2/')
shutil.rmtree('./data/icon_grib2/')
shutil.rmtree('./data/icon_nc/')

FileNotFoundError: [Errno 2] No such file or directory: './data/icon_bz2/'

In [9]:
for time in icon_times:
    for var in icon_variables:
        _ = from_icon_to_ru_nc(
            path_to_nc=Path(f'./data/icon_nc'),
            var=var,
            icon_time=time,
            path_to_result=Path(f'./data/forecast/{time}'))
